# MNIST by Convolution NN (LeNet-5)
* 2 conv([5,5],64) 2 maxpooling(strides=2) FC(1024)
* input=[batch,length,width,deep]
* Convolution kernel(w) = [length,width,deep_in,deep_out]
* strides = [1,step,step,1]
* new_height = new_width = (input — kernel + 1) / Strides

In [1]:
#import data and library
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images  # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
#def feature map and pooling


def weight_variable(name,shape):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.get_variable(name,shape,initializer=initializer,dtype=tf.float32)

def bias_variable(name,shape):
    return tf.get_variable(name, shape)

def conv2d(inputs, w):
    return tf.nn.conv2d(inputs,w,strides = [1,1,1,1],padding = 'SAME')

def max_pooling_2x2(inputs,x=2,y=2):
    ksize = [1,x,y,1]
    return tf.nn.max_pool(inputs,ksize,strides = [1,2,2,1],padding = 'SAME')

In [ ]:
#set outlines 
inputs_num = 784 # 28*28 =784  (784,1)
outputs_num = 10 # 0 to 9   

#reset variable in tensorflow
tf.reset_default_graph()

#set placeholder(input and target)
inputs= tf.placeholder(tf.float32,[None,inputs_num],)
targets= tf.placeholder(tf.float32,[None,outputs_num],)
#transfer image to 4D data([batch,in_height,in_width,in_channels])
inputs_image = tf.reshape(inputs,[-1,28,28,1])

#set 1st convolution layer w and b
w_conv1 = weight_variable('w_1',[5,5,1,32])
b_conv1 = bias_variable('b_1',[32])
#set 2nd convolution layer w and b
w_conv2 = weight_variable('w_2',[5,5,32,64])
b_conv2 = bias_variable('b_2',[64])

#convolution layer and max pooling (relu activation function) 2 times pooling 28 ->14 ->7
h_conv1 = tf.nn.relu(conv2d(inputs_image,w_conv1)+b_conv1)
h_pool1 = max_pooling_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2 = max_pooling_2x2(h_conv2)

#set 1st fully connected variable(last layer 7*7*64, 1024 nuron)
fc_w1 = weight_variable('fc_w1',[7*7*64,1024])
fc_b1 = bias_variable('fc_b1',[1024])

#flatten
h_pool2_flatten = tf.reshape(h_pool2,[-1,7*7*64])

# fully connected 1
fc_1 = tf.nn.relu(tf.matmul(h_pool2_flatten,fc_w1)+fc_b1)

# dropout(testing need turn off)
keep_prop = tf.placeholder(tf.float32)
fc_1_drop = tf.nn.dropout(fc_1,keep_prop)

#set 2nd fully connected variable(last layer 1024, 10 nuron)
fc_w2 = weight_variable('fc_w2',[1024,10])
fc_b2 = bias_variable('fc_b2',[10])

#predicted outputs
outputs = tf.nn.softmax(tf.matmul(fc_1_drop,fc_w2)+fc_b2)

# set loss and optimize object
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate=0.003,).minimize(loss)

#after softmax, let probability to 0 or 1 by argmax function, and judge between target and output
out_eql_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

#predicte accuracy (need cast to float32)
accuracy = tf.reduce_mean(tf.cast(out_eql_target,tf.float32))

#框架完成

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)
#sess
sess= tf.InteractiveSession()

#initial variable
initializer = tf.global_variables_initializer()
sess.run(initializer)


#set training paramerter
epochs = 1
batch_num = 1000
batch_cycle = mnist.train._num_examples // batch_num
prev_validation = 9999999.

#Training model
import time
start = time.time()
for epoch_counter in range(epochs):
    curr_epoc_loss = 0.
    for batch_counter in range(batch_cycle):
        input_batch, target_batch = mnist.train.next_batch(batch_num)
        _,batch_loss = sess.run([optimizer,loss],
                                feed_dict={inputs:input_batch,targets: target_batch,keep_prop:0.9})
        curr_epoc_loss += batch_loss
    curr_epoc_loss/=batch_cycle
    # validation
    input_batch, target_batch = mnist.validation.next_batch(100)
    val_loss, val_accuracy = sess.run([loss,accuracy],feed_dict={inputs:input_batch,targets:target_batch, keep_prop:0.8})
    if val_loss>prev_validation:
        break
    prev_validation = val_loss
    print(f'Epoch cycle: {epoch_counter + 1}'+
         f' Training loss: {curr_epoc_loss:.3f}'+
         f' Validation loss: {val_loss:.3f}'+
         f' Validation accuracy: {val_accuracy*100.:.2f}'+'%'+
         f' times: {time.time() - start}')

    #TEST
    input_batch, target_batch = mnist.test.next_batch(100)
    test_accuracy = sess.run([accuracy],
                             feed_dict ={inputs:input_batch,targets:target_batch,keep_prop:1})
    test_accuracy_percent=test_accuracy[0]*100.
    print(f'test accuracy: {test_accuracy_percent:.2f}')
print('end of training')


E:\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [38]:
tf.InteractiveSession.close(sess)

In [ ]:
#CNN model

import numpy as np
import tensorflow as tf

def weight_variable(shape):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.get_variable('weight',shape,initializer=initializer,dtype=tf.float32)

def bias_variable(shape):
    initialier = tf.constance_initializer()
    return tf.get_variable('bias',shape,initializer=initializer,dtype=tf.float32)

def conv2d(inputs, w):
    return tf.nn.conv2d(inputs,w,strides = [1,1,1,1],padding = 'SAME')

def max_pooling_2x2(inputs,x=2,y=2):
    ksize = [1,x,y,1]
    return tf.nn.max_pool(inputs,ksize,strides = [1,2,2,1],padding = 'SAME')

class MNISTcnn(object):
    def __init__(self,inputs,targets,conf):
        self.inputs = inputs
        self.targets = targets
        self.keep_drop = tf.palceholder(tf.float32)
        
        # 1st layer
        with tf.variable_scope(conv1):
            W_conv1 = weight_variable([5, 5, 1, 32])
            b_conv1 = bias_variable([32])
            h_conv1 = tf.nn.relu(conv2d(self.inputs, W_conv1) + b_conv1)
            h_pool1 = max_pool_2x2(h_conv1)
        # 2nd layer
        with tf.variable__scope(conv2):
            W_conv2 = weight_variable([5, 5, 32, 64])
            b_conv2 = bias_variable([64])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
            h_pool2 = max_pool_2x2(h_conv2)
        #fully connected 1st layer
        with tf.variable_scope("fc1"):
            shape = int(np.prod(h_pool2.get_shape()[1:]))
            W_fc1 = weight_variable([shape, 1024])
            b_fc1 = bias_variable([1024])
            h_pool2_flat = tf.reshape(h_pool2, [-1, shape])
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        # dropout
        h_fc1_drop = tf.nn.dropout(h_fc1, self.keep_prob)
        #fully connected to outputs
        with tf.variable_scope("fc2"):
            W_fc2 = weight_variable([1024, 10])
            b_fc2 = bias_variable([10])
            outputs = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.targets, logits=outputs))
        self.pred = tf.argmax(outputs, 1)

        self.correct_prediction = tf.equal(tf.argmax(outputs,1), tf.argmax(self.targets,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))